## Open Government Data, provided by **OpenDataZurich**
*Autogenerated Python starter code for data set with identifier* **geo_gebietseignung_fuer_leitungsgebundene_energieversorgung**

## Dataset
# **Gebietseignung für leitungsgebundene Energieversorgung**

## Description

Der Geodatensatz zeigt im Hektarraster die langfristige Gebietseignung für die leitungsgebundene Energieversorgung der Stadt Zürich.

**Zweck**: Die Analyse der langfristigen Gebietseignung ist zentral für die Bestimmung des Perimeters von neuen oder von möglichen Erweiterungen bestehender leitungsgebundener Energieversorgungen (Fernwärme, Gas).

**Genereller Hinweis zum Geodatensatz:**


Weitere Informationen zu diesem Datensatz finden sie unter [«**Gebietseignung für leitungsgebundene Energieversorgung»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/a376d9e8-3a9a-4f3c-82fb-b183fe086335).				
			   

**Informationen zum Datensatz:**

Rechtliche Grundlage: STRB 1077 / 2016
Planungsbericht Energieversorgung und Massnahmenkatalog (Massnahme E12)
Zuständige Stelle technische Bearbeitung während Erarbeitung Energieplanung: GeoZ
Zuständige Stelle technische Bearbeitung während laufender Nachführung Energieplanung: GeoZ

**Datenerfassung:**

Die Ermittlung der Gebietseignung basiert auf den Kriterien, welche in Tab. 2 des Planungsberichts «Energieversorgung» erläutert werden.

**Datengrundlage:**

-	Erwarteter Nutzwärmebedarf für Raumwärme und Warmwasser 2050 gemäss Effizienzszenario (Konzept Energieversorgung 2050)
-	Strassen nach Eigentümer (Datei 150127_Strassen_Eigentum.shp, TAZ)
-	Geleise gemäss kantonalem Richtplan (Normalspur, Schmalspur, Anschlussgeleise, inkl. projektierte, aber ohne Tunnels)
-	Trolleybuslinien (Datei 150420_VBZ_Buslinien)
-	Wärmenutzungsatlas des Kantons Zürich (Zulässigkeit von Erdwärmesonden)
-	Verdichtungsräume gemäss der räumlichen Entwicklungsstrategie der Stadt Zürich (Datei AFS_Verdichtungsraeume_Stand_20150122)
-	Bestehende Fernwärmegebiete gemäss Energieplanung 2016
-	Archäologische Verdachtsflächen
-	Erwarteter Kältebedarf 2050
-	Gebiete mit Arealüberbauung oder Sondernutzungsplanung (Quelle: AfS 2016)
-	Anzahl Energiebereitstellungsanlagen pro Hektare gemäss Statistik Zürich (SSZ)

**Weitere Informationen zu diesem Thema:**

Finden Sie unter diesem [**Link**](https://www.stadt-zuerich.ch/content/dib/de/index/energieversorgung/energieplanung.html)

**Statisches Vorschaubild:**

![BildText](https://www.gis.stadt-zuerich.ch/zueriplan_docs/geocat/a376d9e8-3a9a-4f3c-82fb-b183fe086335.png)



## Data set links

[Direct link by OpenDataZurich for dataset](https://data.stadt-zuerich.ch/dataset/geo_gebietseignung_fuer_leitungsgebundene_energieversorgung)

https://www.stadt-zuerich.ch/geodaten/download/Gebietseignung_fuer_leitungsgebundene_Energieversorgung?format=geojson_link<br>


## Metadata
- **Publisher** `Energiebeauftragte, Departementssekretariat DIB, Departement der Industriellen Betriebe`
- **Maintainer** `Open Data Zürich`
- **Maintainer_email** `opendata@zuerich.ch`
- **Keywords** `Energie`
- **Tags** `['energieplanung', 'geodaten', 'geoportal', 'leitungsbaukosten', 'leitungsgebundene-energieversorgung', 'polygondaten', 'prognose', 'stzh', 'vektordaten']`
- **Metadata_created** `2023-11-06T03:25:48.677246`
- **Metadata_modified** `2025-12-20T06:39:26.965681`


## Imports and helper functions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 

import json
import xml.etree.ElementTree as ET
import requests
import geopandas as gpd
import re

In [ ]:
# helper function for reading datasets with proper separator
def get_dataset(url):
    # get URL ofs WFS service
    dataset_identifier = re.search(r'\/([^\/\?]+)\?', url).group(1)
    url_geoportal = f"https://www.ogd.stadt-zuerich.ch/wfs/geoportal/{dataset_identifier}"
    print("Getting available layers from:", url_geoportal)
    
    # Parameter for GetCapabilities
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetCapabilities"
    }
    
    # send GetCapabilities
    response = requests.get(url_geoportal, params=params)

    # parse XML answer
    root = ET.fromstring(response.content)

    # define Namespace 
    namespace = {'wfs': 'http://www.opengis.net/wfs'}
    
    # Exctract available layers
    layers = [feature_type.find('wfs:Name', namespace).text for feature_type in root.findall('.//wfs:FeatureType', namespace)]
    
    print("Available layers:", layers)
    print("First layer is set as default. To chose another layer set it as typename in the get_dataset() function.")

    # set first layer als typename
    typename = layers[0]
    print("Chosen typename:", typename)

    # Parameter GetFeature request
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetFeature",
        "typename": typename,
        "outputFormat": "application/json"
    }
    
    # GetFeature request
    response = requests.get(url_geoportal, params=params)

    # Load GeoJSON in GeoDataFrame
    gdf = gpd.read_file(json.dumps(response.json()))
    return gdf

## Load the data

In [ ]:
gdf = get_dataset('https://www.stadt-zuerich.ch/geodaten/download/Gebietseignung_fuer_leitungsgebundene_Energieversorgung?format=geojson_link')

## Analyze the data

In [ ]:
gdf.plot()

In [ ]:
# drop columns that have no values
gdf.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(f'The dataset has {gdf.shape[0]:,.0f} rows (observations) and {gdf.shape[1]:,.0f} columns (variables).')
print(f'There seem to be {gdf.duplicated().sum()} exact duplicates in the data.')

In [ ]:
gdf.info(memory_usage='deep', verbose=True)

In [ ]:
gdf.head()

In [ ]:
# display a small random sample transposed in order to see all variables
gdf.sample(3).T

In [ ]:
# describe non-numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(exclude='number'))
except:
    print("No categorical data in dataset.")

In [ ]:
# describe numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(include='number'))
except:
    print("No numercial data in dataset.")

In [ ]:
# plot a histogram for each numerical feature
try:
    gdf.hist(bins=25, rwidth=.9)
    plt.tight_layout()
    plt.show()
except:
    print("No numercial data to plot.") 

In [ ]:
# continue your code here...

**Contact**: opendata@zuerich.ch